In [410]:
'''
path = 'OpenIE/Stanford-OpenIE/nyt'
f = 'segmentation'

segOutput = open(path +'/train.seg.txt', 'w')

with open(path +'/'+  f + '.txt','r') as thefile:
    for line in thefile:
        pointer=1
        in_phrase = False
        st=[]
        arr = line.rstrip('\n').split(' ')
        for word in arr:
            if '<phrase>' in word:
                in_phrase = True
            if in_phrase :    
                st.append(str(pointer))
            else:
                st.append('O')
            if '</phrase>' in word:
                in_phrase = False
                pointer+=1
        sentence=' '.join(st)
        segOutput.write(sentence + '\n')
segOutput.close()       

#%%

data_all=[]
read_file =['tokens.', 'pos.', 'ner.' , 'dep.', 'seg.']
for f in read_file:
    thefile = open(path + '/train.' + f + 'txt','r')
    i=0
    for line in thefile:
        arr = line.rstrip('\n').split(' ')
        if not data_all:
            data_all= [[] for t in range(235972)]
        if not data_all[i]:
            data_all[i] = [[] for t in range(len(arr))]
        for j,ele in enumerate(arr):
            data_all[i][j].append(ele)
        i=i+1
# merge phrases <Hu Jintao>-><Hu_Jintao>, not PERSON PERSON, Hu PERSON, PERSON Jintao
f='phrase.'
thefile = open(path + '/train.' + f + 'txt','w')
merged_data=[[] for i in range(len(data_all))]
for j, sentence in enumerate(data_all):
    for i, word in enumerate(sentence):
        newWord=word[0]
        if (word[2] == 'O' or word[2] == 'MISC') and word[4] =='O' or i==0:
            merged_data[j].append(newWord)
            continue
        else:
            if word[4] !='O' and word[4] !=sentence[i-1][4] and (word[2] == 'O' or word[2] == 'MISC'):
                merged_data[j].append(newWord)
            elif  word[4] !='O' and word[4] ==sentence[i-1][4] or (word[2] != 'O' and word[2] != 'MISC' and word[2] ==sentence[i-1][2]):
                merged_data[j][-1]='_'.join((merged_data[j][-1],newWord))
            elif word[2] != 'O' and word[2] != 'MISC' and word[2] !=sentence[i-1][2] :
                newWord='_'.join((word[2],newWord))
                merged_data[j].append(newWord)
    merged_data[j]=' '.join(merged_data[j])
    thefile.write(merged_data[j]+'\n')
''' 

"\npath = 'OpenIE/Stanford-OpenIE/nyt'\nf = 'segmentation'\n\nsegOutput = open(path +'/train.seg.txt', 'w')\n\nwith open(path +'/'+  f + '.txt','r') as thefile:\n    for line in thefile:\n        pointer=1\n        in_phrase = False\n        st=[]\n        arr = line.rstrip('\n').split(' ')\n        for word in arr:\n            if '<phrase>' in word:\n                in_phrase = True\n            if in_phrase :    \n                st.append(str(pointer))\n            else:\n                st.append('O')\n            if '</phrase>' in word:\n                in_phrase = False\n                pointer+=1\n        sentence=' '.join(st)\n        segOutput.write(sentence + '\n')\nsegOutput.close()       \n\n#%%\n\ndata_all=[]\nread_file =['tokens.', 'pos.', 'ner.' , 'dep.', 'seg.']\nfor f in read_file:\n    thefile = open(path + '/train.' + f + 'txt','r')\n    i=0\n    for line in thefile:\n        arr = line.rstrip('\n').split(' ')\n        if not data_all:\n            data_all= [[] for

In [1]:
# sentence tree class
from collections import deque
class SnTree(object):
    #global cc_words
    #cc_words=['and','or','but']
    def __init__(self, sentence_tree):
        #self.order = []     #visited order 
        self.tree = sentence_tree
        self.root=-1
        self.subtrees={}
        self.treeEncode = {}
        
    def set_root(self, r_id):
        self.root=r_id
           
            
    def encode_tree(self, r, coding):
        self.treeEncode[r]=coding
        #print(r, self.tree[r][-1])                
        for i,p in enumerate(self.tree[r][-1]):
            #print(p)
            new_coding=coding+chr(ord('A') + i)
            self.encode_tree(r=p,coding=new_coding)

            

    
    def split_conj(self):
        search_queue = deque()
        search_queue += list(self.subtrees.items())
        #print('seachque',search_queue)
        # split on each subtree
        while search_queue:
            tempSen=list(search_queue.popleft()[1])
            sentence = sorted(tempSen)
            #print('sentence',sentence)
            not_position= -1
            neither_position=-1
            for i,word in enumerate(sentence):
                if type(word)==float:
                    break
                #if two vb conjunction
                #second vb's dep is conj and its parent is verb
                if self.tree[word][5]=='not': 
                    not_position=word
                if self.tree[word][5]=='neither':
                    neither_position=word
                
                if self.tree[word][2]=='VERB' and self.tree[word][3]=='conj' and self.tree[self.tree[word][4]][2]=='VERB' and self.tree[word][4] in sentence:
                    verb1=self.tree[word][4]
                    verb2=word
                    break_point=-1
                    
                    difSub=False 
                    difObj=False
                    noObj1=True
                    noObj2=True
                    for w2 in self.tree[word][-1]:
                        if 'nsubj' in self.tree[w2][3]: #do not share the same subject
                            difSub=True
                            #print('found subject for the second verb', w2)
                        if self.tree[w2][3]=='dobj':
                            noObj2=False #has object
                        

                    for w1 in self.tree[self.tree[word][4]][-1]:
                        
                        if self.tree[w1][3]=='dobj':
                            noObj1=False #has object
                            if w1<word: #do not share the same object
                                difObj = True
                            #print('found object for the first verb',w1)
                        if w1>verb1 and w1<word and (self.tree[w1][2]=='ADV' or self.tree[w1][2]=='ADP'):
                            difObj = True
                        if self.tree[w1][3]=='cc' and w1<word and w1 in sentence and w1>verb1:
                            break_point=w1
                            #print('break point', break_point)
                        
                    while(False):        
                    #while(i>0 and break_point<0):
                        i-=1
                        if self.tree[sentence[i]][1]==',' or self.tree[sentence[i]][1]==';':
                            break_point=sentence[i]
                            #print('break point', break_point)
                    if break_point<0:
                        break
                    sent1=[]
                    sent2=[]
                    #if difSub and (difObj or (noObj1 and noObj2)):
                    if difSub :
                        #not share subject or object. safe to split
                        #split at cc
                        for w in sentence:
                            if w < break_point:
                                sent1.append(w)
                            elif w == break_point:
                                continue
                            else:
                                sent2.append(w)
                    #elif difSub==False and (difObj or (noObj1 and noObj2)):
                    else:
                        #same subject, different object
                        for j,w in enumerate(sentence):
                            if w < verb1:
                                if (w == not_position and self.tree[sentence[j+1]][1]=='only') or ( sentence[j-1]== not_position and self.tree[w][1]=='only'):
                                    #not only, but also
                                    continue
                                if self.tree[w][1]=='both' or self.tree[w][1]=='either':
                                    #not only, but also
                                    continue
                                elif w == not_position and self.tree[break_point][1] == 'but' or (self.tree[w][3]=='advmod' and self.tree[w][4]==verb1 and sentence.index(verb2)-sentence.index(break_point)>1):
                                    #not , but
                                    # adverb
                                    sent1.append(w)
                                elif w == not_position and self.tree[break_point][1] == 'or':
                                    #not , or
                                    sent1.append(w)
                                    sent2.append(w)
                                elif w == neither_position:
                                    #neither_nor/or
                                    sent1.append(w+0.5)
                                    sent2.append(w+0.5)
                                else:
                                    sent1.append(w)
                                    sent2.append(w)
                            elif w<break_point:
                                sent1.append(w)
                            elif w==break_point:
                                continue
                            else:
                                if (self.tree[w][1]=='also' and self.tree[break_point][1]=='but' and sentence[j-1]==break_point):
                                    continue
                                sent2.append(w)
                                
                    
                    
                    self.subtrees[verb1]=set(sent1)
                    self.subtrees[verb2]=set(sent2)
                    #elif difSub :
                    #print(verb2, sent2)
                    search_queue+= [(verb2, set(sent2)) ]  
                    break    
                    
                #second vb's dep is conj and its parent's dep is acomp  
                #if word[1][3]=='conj' and self.tree[word[1][4]][3]=='acomp':

    
    def split_noun(self, r):
        #print('start')
        if r !=-1:
            search_queue = deque()
            search_queue.append(r)
            self.subtrees[r]=set()
            #print(search_queue)
        else:
            print('root is None')
            return -1

        while search_queue:
            person = search_queue.popleft()
            #print(person)

            if (not person in self.subtrees[r]) and (person in self.tree.keys()):
                if (self.tree[person][2] != 'NOUN' and self.tree[person][2] != 'PROPN') or person ==r :
                    search_queue += self.tree[person][-1]
                    self.subtrees[r].add(person)
                else:
                    self.subtrees[r].add(person)
                    self.split_noun(person)
    
    def concentrate_subtrees(self,windowsize_min,windowsize_max):
        keys=list(self.subtrees.keys())
        for k in reversed(keys):
            if len(self.subtrees[k]) < windowsize_min:
                #print(k,self.subtrees[k])
                for key in self.subtrees:
                    if k in self.subtrees[key]:
                        #print(key, self.subtrees[key])
                        mergeLen=len(self.subtrees[key])+ len(self.subtrees[k])
                        if mergeLen<windowsize_max or len(self.subtrees[k])<3:
                            self.subtrees[key].update(self.subtrees[k])
                            if k!=key:
                                del self.subtrees[k]
                        break
                        
    
    def subsentecne(self):
        subs=[]
        #print(self.subtrees)
        for key in self.subtrees.keys():
        #for key in sorted(self.subtrees.keys()):
            subs.append([])
            sentence_by_id = sorted(list(self.subtrees[key]))
            subs[-1].append(self.tree[key][5])
            subs[-1].append('\t')
            for w in sentence_by_id:
                if type(w) == float:
                    subs[-1].append('not')
                else:
                    if 'CHEMICAL' in self.tree[w][1] or 'GENE' in self.tree[w][1] or 'SPECIES' in self.tree[w][1] or 'DISEASE' in self.tree[w][1]:
                        subs[-1].append(self.tree[w][1])
                    else:
                        subs[-1].append(self.tree[w][5])
                #print(self.tree[w][1])
            subs[-1]=' '.join(subs[-1])
        return subs
    def subpos(self):
        subs=[]
        for key in self.subtrees.keys():
        #for key in sorted(self.subtrees.keys()):
            subs.append([])
            sentence_by_id = sorted(list(self.subtrees[key]))
            subs[-1].append(self.tree[key][2])
            subs[-1].append('\t')
            for w in sentence_by_id:
                if type(w) == float:
                    subs[-1].append('not')
                else:
                    subs[-1].append(self.tree[w][2])
                
            subs[-1]=' '.join(subs[-1])
        return subs
    
    def subencode(self):
        #print(self.treeEncode)
        subs=[]
        #for key in sorted(self.subtrees.keys()):
        for key in self.subtrees.keys():    
            subs.append([])
            sentence_by_id = sorted(list(self.subtrees[key]))
            subs[-1].append(self.treeEncode[key])
            #print(self.treeEncode[key])
            subs[-1].append('\t')
            for w in sentence_by_id:
                if type(w) == float:
                    subs[-1].append(self.treeEncode[key]+'A')
                else:
                    subs[-1].append(self.treeEncode[w])
                    #print(self.treeEncode[w])
                
            subs[-1]=' '.join(subs[-1])
        return subs
    

In [21]:

import spacy
subSentence=[]
sentence_tree=dict()
nlp = spacy.load('en_core_web_sm')
j=198728
#doc = nlp(merged_data[j])
line='Pre_treatment of CHEMICAL can decrease the overexpression of GENE and GENE induced by CHEMICAL'
doc=nlp(line)
print(line)
for i, token in enumerate(doc):
    sentence_tree[token.idx]=[i, token.text,token.pos_, token.dep_,token.head.idx,token.lemma_, [child.idx for child in token.ancestors], [child.idx for child in token.children]]
    if token.dep_ =='ROOT':
        root_id=token.idx
    print(token.idx, token.text,token.pos_, token.dep_,token.head.idx,token.lemma_, [child.idx for child in token.ancestors],[child.idx for child in token.children])

sentence_try1=SnTree(sentence_tree)
sentence_try1.root=root_id
sentence_try1.encode_tree(root_id,'A')

sentence_try1.split_noun(root_id)
sentence_try1.split_conj()
#sentence_try1.concentrate_subtrees(4,10)
subSentence += sentence_try1.subsentecne()
subSentence += sentence_try1.subpos()
subSentence += sentence_try1.subencode()
print(subSentence)

from spacy import displacy

svg = displacy.render(doc, style='dep')
output_path = open('parse.svg','w', encoding='utf-8')
output_path.write(svg)
output_path.close()


Pre_treatment of CHEMICAL can decrease the overexpression of GENE and GENE induced by CHEMICAL
0 Pre_treatment PROPN nsubj 30 pre_treatment [30] [14]
14 of ADP prep 0 of [0, 30] [17]
17 CHEMICAL PROPN pobj 14 chemical [14, 0, 30] []
26 can VERB aux 30 can [30] []
30 decrease VERB ROOT 30 decrease [] [0, 26, 43]
39 the DET det 43 the [43, 30] []
43 overexpression NOUN dobj 30 overexpression [30] [39, 58]
58 of ADP prep 43 of [43, 30] [61]
61 GENE PROPN pobj 58 gene [58, 43, 30] [66, 70]
66 and CCONJ cc 61 and [61, 58, 43, 30] []
70 GENE PROPN conj 61 gene [61, 58, 43, 30] [75]
75 induced VERB acl 70 induce [70, 61, 58, 43, 30] [83]
83 by ADP agent 75 by [75, 70, 61, 58, 43, 30] [86]
86 CHEMICAL PROPN pobj 83 chemical [83, 75, 70, 61, 58, 43, 30] []
['decrease \t pre_treatment can decrease overexpression', 'pre_treatment \t pre_treatment of CHEMICAL', 'chemical \t CHEMICAL', 'overexpression \t the overexpression of GENE', 'gene \t GENE and GENE', 'gene \t GENE induce by CHEMICAL', 'chemi

In [4]:
'''# bio data replace CHEMICAL,GENE,...
path = 'OpenIE/Stanford-OpenIE/bio'
merged_data_dict=dict()
merged_data=[]
read_file =['ner.']
entity_dict=dict()
entity_id=0
for f in read_file:
    thefile = open(path + '/train.' + f + 'txt','r')
    for line in thefile:
        sn = line.rstrip('.\n')
        if len(sn)<2:
            continue
        else:
            
            arr=sn.split(' ')
            tokens=[]
            j=0
            for i in range(len(arr)):
                if 'CHEMICAL' in arr[i] or 'GENE' in arr[i] or 'SPECIES' in arr[i] or 'DISEASE' in arr[i]:
                    punct=''
                    if not arr[i][-1].isalpha():
                        punct=arr[i][-1]
                    centity=arr[i].split('-')
                    original_entity='-'.join(centity[0:-1])
                    temp_entity=centity[0].split('_')[0] + str(j)
                    entity_dict[temp_entity]=original_entity
                    if len(centity)>1:
                        arr[i]=temp_entity + '_'+centity[-1]
                    else:
                        arr[i]=temp_entity + punct
                    
                    tokens.append(arr[i])
                    j+=1
                elif '-' in arr[i]:
                    t = arr[i].replace('-','_')
                    tokens.append(t)
                else:
                    tokens.append(arr[i])
            merged_data.append(' '.join(tokens))
'''

In [ ]:
# bio tag data replace CHEMICAL,GENE,...
path = 'bio2'
merged_data_dict=dict()
merged_data=[]
read_file =['ner.']
ner_types=['DISEASE','GENE', 'CHEMICAL','PATHWAY','SNP','BP','MF','CC']
entity_dict=dict()
entity_id=0
j=0
line_number=0
line_id=[]
for f in read_file:
    thefile = open(path + '/corpus_tagged.txt','r')
    for line in thefile:
        line_number=line_number+1
        sn = line.rstrip('.\n')           
        arr=sn.split(' ')
        tokens=[]
        if len(arr)<5:
            continue
        for i in range(len(arr)):
            for ner in ner_types:
                start_with_dash=False
                if ner in arr[i]:
                    punct=''
                    if arr[i][-2:]=='),' or arr[i][-2:]==');':
                        punct=arr[i][-2:]
                    elif arr[i][-1]==',' or arr[i][-1]==')' or arr[i][-1]=='"' or arr[i][-1]==';':
                        punct=arr[i][-1]

                    centity=arr[i].split('-')
                    if not centity[0]:
                        start_with_dash=True
                        centity.pop(0)
                    temp_entity=centity[0].split('_')[0] + str(j)

                    if ner not in temp_entity:
                        for nerner in ner_types:
                            if nerner in temp_entity:
                                temp_entity=temp_entity.replace(nerner,ner)

                    t1=ner+ str(j)


                    if len(centity)>1:
                        original_entity='-'.join(centity[0:-1])
                        arr[i]=temp_entity + '_'+centity[-1]
                    else:
                        arr[i]=temp_entity + punct
                        if start_with_dash:
                            arr[i]='-'+arr[i]
                        original_entity=centity
                    entity_dict[t1]=original_entity


                    j+=1
                    break
            if '-' in arr[i]:
                t = arr[i].replace('-','_')
                tokens.append(t)
            else:
                tokens.append(arr[i])
        merged_data.append(' '.join(tokens))
        line_id.append(str(line_number))
print('reading_in:done')

In [1]:
# bio data replace CHEMICAL,GENE,...
path = 'bio'
merged_data_dict=dict()
merged_data=[]
read_file =['ner.']
ner_types=['DISEASE','GENE', 'CHEMICAL','SPECIES']
entity_dict=dict()
entity_id=0
j=0
line_number=0
line_id=[]
for f in read_file:
    thefile = open(path + '/train1.' + f + 'txt','r')
    for line in thefile:
        line_number=line_number+1
        sn = line.rstrip('.\n')           
        arr=sn.split(' ')
        tokens=[]
        if len(arr)<5:
            continue
        for i in range(len(arr)):
            for ner in ner_types:
                start_with_dash=False
                if ner in arr[i]:
                    punct=''
                    if arr[i][-2:]=='),' or arr[i][-2:]==');':
                        punct=arr[i][-2:]
                    elif arr[i][-1]==',' or arr[i][-1]==')' or arr[i][-1]=='"' or arr[i][-1]==';':
                        punct=arr[i][-1]

                    centity=arr[i].split('-')
                    if not centity[0]:
                        start_with_dash=True
                        centity.pop(0)
                    temp_entity=centity[0].split('_')[0] + str(j)

                    if ner not in temp_entity:
                        for nerner in ner_types:
                            if nerner in temp_entity:
                                temp_entity=temp_entity.replace(nerner,ner)

                    t1=ner+ str(j)


                    if len(centity)>1:
                        original_entity='-'.join(centity[0:-1])
                        arr[i]=temp_entity + '_'+centity[-1]
                    else:
                        arr[i]=temp_entity + punct
                        if start_with_dash:
                            arr[i]='-'+arr[i]
                        original_entity=centity
                    entity_dict[t1]=original_entity


                    j+=1
                    break
            if '-' in arr[i]:
                t = arr[i].replace('-','_')
                tokens.append(t)
            else:
                tokens.append(arr[i])
        merged_data.append(' '.join(tokens))
        line_id.append(str(line_number))
print('reading_in:done')

reading_in:done


In [3]:
import pickle
g = 'extraction_dictionary.'
path = 'bio'
ex_id='/train2.'
entityOutput = path +ex_id+ g + 'pickle'
with open(entityOutput, 'wb') as handle:
    pickle.dump(entity_dict, handle)

In [13]:
import pickle
import spacy
nlp = spacy.load('en_core_web_sm')
path = 'bio'
write=True
ex_id='/train2.'
f = 'subSentence.'
subOutput = open(path +ex_id+ f + 'txt', 'w')
'''
g = 'extraction_dictionary.'
entityOutput = path +ex_id+ g + 'pickle'
with open(entityOutput, 'wb') as handle:
    pickle.dump(entity_dict, handle)
'''
h = 'subPOS.'
posOutput = open(path +ex_id+ h + 'txt', 'w')
l = 'subEncode.'
encodeOutput = open(path +ex_id+ l + 'txt', 'w')

subSentence=[]
subSentence_pos=[]
subSentence_encode=[]
for j in range(len(merged_data)):
#    j=17
    doc = nlp(merged_data[j])
    sentence_tree={}
    sentence_visit=[]
    for i, token in enumerate(doc):
        sentence_tree[token.idx]=[i, token.text,token.pos_, token.dep_,token.head.idx,token.lemma_, [child.idx for child in token.ancestors],[child.idx for child in token.children]]
        if token.dep_ =='ROOT':
            root_id=token.idx
        #print(token.text, token.idx, token.dep_, token.head.text, token.pos_,[child.idx for child in token.children])

    sentence_try=SnTree(sentence_tree)

    sentence_try.root=root_id
    sentence_try.encode_tree(root_id,'A')
    sentence_try.split_noun(root_id)
    sentence_try.split_conj()
    #sentence_try.concentrate_subtrees(4,10)
    #pprint.pprint(sentence_try.subtrees)
    subSentence += sentence_try.subsentecne()
    if j % 2000 == 0:
        print(j)
    subSentence_pos += sentence_try.subpos()
    subSentence_encode += sentence_try.subencode()
    #print(subSentence)
    if write:
        subOutput.write(line_id[j] + '\n')
        for subs in sentence_try.subsentecne():
            subOutput.write(subs + '\n')        
        subOutput.write('\n')
        posOutput.write(line_id[j] + '\n')
        for subs in sentence_try.subpos():
            posOutput.write(subs + '\n')        
        posOutput.write('\n')
        encodeOutput.write(line_id[j] + '\n')
        for subs in sentence_try.subencode():
            encodeOutput.write(subs + '\n')        
        encodeOutput.write('\n')
        
subOutput.close()
posOutput.close()
encodeOutput.close()


0
2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000
24000
26000
28000
30000
32000
34000
36000
38000
40000
42000
44000
46000
48000
50000
52000
54000
56000
58000
60000
62000
64000
66000
68000
70000
72000
74000
76000
78000
80000
82000
84000
86000
88000
90000
92000
94000
96000
98000
100000
102000
104000
106000
108000
110000
112000
114000
116000
118000
120000
122000
124000
126000
128000
130000
132000
134000
136000
138000
140000
142000
144000
146000
148000
150000
152000
154000
156000
158000
160000
162000
164000
166000
168000
170000
172000
174000
176000
178000
180000
182000
184000
186000
188000
190000
192000
194000
196000
198000
200000
202000
204000
206000
208000
210000
212000
214000
216000
218000
220000
222000
224000
226000
228000
230000
232000
234000
236000
238000
240000
242000
244000
246000
248000
250000
252000
254000
256000
258000
260000
262000
264000
266000
268000
270000
272000
274000
276000
278000
280000
282000
284000
286000
288000
290000
292000
294000
296000
298000
300000


In [ ]:
'''
from collections import defaultdict
import numpy as np
import multiprocessing
import threading
import collections
import math

ARR = multiprocessing.Array('b', 235969 + 1)
DEBUG = False
MIN_SUP = 5

candidates=[]
current_words = set()
words_freq = defaultdict(int)

subOutput = open(path +'/train.' + f + 'txt', 'r')
snID=0
for line in subOutput:
    sn = line.rstrip(' .\n').split(' \t ')
    snID+=1
    if len(sn)<2:
        continue
    else:
        temp=dict()
        temp['sent_id']=snID
        arr=sn[1].split(' ')
        for i in range(len(arr)):
            if 'CHEMICAL' in arr[i] or 'GENE' in arr[i] or 'SPECIES' in arr[i] or 'DISEASE' in arr[i]:
                arr[i]=arr[i].split('_')[0]
                words_freq[arr[i]] += 1
                current_words.add(arr[i])
            
            else:
                arr[i]=arr[i].lower()
                words_freq[arr[i]] += 1
                current_words.add(arr[i])
        temp['sent']=arr
        candidates.append(temp)
'''

In [ ]:
'''
def is_subseq(sequence, subsequece):
    if len(sequence) < len(subsequece):
        return False
    it = iter(sequence)
    return all(c in it for c in subsequece)

def flatten(l):
    return list(itertools.chain.from_iterable(l))

def flatten2(l):
    for el in l:
        if isinstance(el, collections.Iterable) and not isinstance(el, (int, str, bytes)):
            yield from flatten2(el)
        else:
            yield el



def match_sent(candidates):
    matched_sents = set()
    match_target = get_subseq_to_match(candidate['sent'])
    for j, to_match in enumerate(candidates):
        if is_subseq(to_match['expanded'], match_target):
            matched_sents.add(to_match['sent_id'])
            if len(matched_sents) > MIN_SUP:
                break
    

def match(candidates, words_freq, original_candidates):
    print(len(candidates))
    global arr
    patterns = defaultdict(set)
    linked = set()
    result = []

    # main loop for MATCH
    # for cnt, candidate in tqdm(enumerate(candidates)):
    for cnt, candidate in enumerate(candidates):
        if DEBUG and cnt == 10:
            break

        if cnt % 100 == 0:
            print(threading.current_thread().name, time.strftime("%m-%d %H:%M:%S"), cnt)
        # if candidate['sent_id'] in linked:
        #     continue
        if arr[candidate['sent_id']] != 0:
            continue

        freq = [words_freq[s] for s in candidate['sent']]
        arg_freq_sort = np.argsort(freq)
        # print(candidate)
        # print(freq)
        # print(arg_freq_sort)

        matched = False
        for replace_pos in arg_freq_sort:
            if matched:
                break
                    
            #for i in range(5):
            for i in [0]: 
                # match
                matched_sents = set()
                matched_pairs = defaultdict(int)

                match_target = candidate['sent']
                for j, to_match in enumerate(original_candidates):
                    if is_subseq(to_match['sent'], match_target):

                        matched_sents.add(to_match['sent_id'])
                        # adding count
                        
                        # if len(matched_sents) > MIN_SUP:
                        #     break

                # print('matched', matched_sents)
                # ***** NOT MATCHING - START REPLACE *****
                if len(matched_sents) < MIN_SUP:
                    # print('replace_pos', replace_pos)
                    # print(candidate['sent'])
                    candidate['sent'][replace_pos]=''
                    
                    
      
                else:
                    matched = True
                    # print(candidate['sent'])
                    for s in range(len(candidate['sent'])):
                        if candidate['sent'][s]=='':
                            candidate['sent'][s]='*'
                    key = ' '.join([s for s in candidate['sent']])
                    if key not in patterns:
                        patterns[key]=matched_sents
                    # linked.union(matched_sents)

                    
                        cur_pattern = {
                            'pattern': key,
                            'freq': len(patterns[key]),
                            'matched': patterns[key],
                        }
                        result.append(cur_pattern)
                    break

    return result
def run_parallel(
    candidates,
    words_freq,
    num_workers = 1,
    save = False,
):
    num_workers+=1
    pool = multiprocessing.Pool(processes=num_workers)

    num_lines = len(candidates)
    batch_size = math.floor(num_lines/(num_workers-1))
    print("batch_size: %d" % batch_size)

    start_pos = [i*batch_size for i in range(0, num_workers)]

    results = [pool.apply_async(match, args=(
        list(candidates[start:start+batch_size]), 
        words_freq,
        candidates,
    )) for i, start in enumerate(start_pos)]

    results = [p.get() for p in results]
    
    # res={}
    # for r in results:
    #     res.update(r)
    res = flatten(results)
    # print(res)
    if(save):
        json.dump(res, open('pattern_result_test1210.json', 'w'), indent=1)
        # self.save_to_file(res, "probase_local_parallel.p")
# old code
# new code at server
'''

In [ ]:
#run_parallel(candidates, words_freq, num_workers=20, save=True)

In [422]:
'''
def frequentPatternMining(tokens,patternOutputFilename, threshold):
    dict_token = {}

    tokensNumber = len(tokens)
    for i in range(tokensNumber):
        token = tokens[i]
        if token == '$':
            continue
        if token in dict_token:
            dict_token[token].append(i)
        else:
            dict_token[token] = [i]
    print ("# of distinct tokens = ", len(dict_token))

    patternOutput = open(patternOutputFilename, 'w')

    frequentPatterns = []
    patternLength = 1
    while (len(dict_token) > 0):
        if patternLength > 10:
            break
        #print "working on length = ", patternLength
        patternLength += 1
        newDict = {}
        for pattern, positions in dict_token.items():
            occurrence = len(positions)
            if occurrence >= threshold:
                frequentPatterns.append(pattern)
                
                patternOutput.write(pattern + "," + str(occurrence) + "\n")
                for i in positions:
                    if i + 1 < tokensNumber:
                        if tokens[i + 1] == '$':
                            continue
                        newPattern = pattern + " " + tokens[i + 1]
                        if newPattern in newDict:
                            newDict[newPattern].append(i + 1)
                        else:
                            newDict[newPattern] = [i + 1]
        dict_token.clear()
        dict_token = newDict
    patternOutput.close()
    return frequentPatterns

path = 'OpenIE/Stanford-OpenIE/bio'
f = 'subSentence.'
subOutput = open(path +'/train.' + f + 'txt', 'r')
snID=0
tokens=[]
for line in subOutput:
    sn = line.rstrip(' .\n').split(' \t ')
    snID+=1
    if len(sn)<2:
        continue
    else:
        temp=dict()
        temp['sent_id']=snID
        arr=sn[1].split(' ')
        for i in range(len(arr)):
            arr[i]=arr[i].split('_')
            if 'CHEMICAL' in arr[i][0]:
                arr[i][0]='CHEMICAL'
                tokens.append('_'.join(arr[i]))
            elif 'GENE' in arr[i][0]:
                arr[i][0]='GENE'
                tokens.append('_'.join(arr[i]))
            elif 'SPECIES' in arr[i][0] :
                arr[i][0]='SPECIES'
                tokens.append('_'.join(arr[i]))
            elif 'DISEASE' in arr[i][0]:          
                arr[i][0]='DISEASE'
                tokens.append('_'.join(arr[i]))         
            else:
                tokens.append('_'.join(arr[i]))
    tokens.append('$')
#frequentPatterns = frequentPatternMining(tokens, patternOutputFilename, threshold)

f_out=path +'/train.metapattern.txt'
frequentPatterns=frequentPatternMining(tokens,f_out,10)
'''